In [32]:
from jina import Document, DocumentArray
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pandas as pd
import numpy as np

In [33]:
# embeddings_df = pd.read_csv("/home/batman/Desktop/py/cybersecurity-jina/data/embeddings_df_with_details.csv") # doesnt work
embeddings_df = pd.read_csv("/home/batman/Desktop/py/cybersecurity-jina/data/dataset_2_embeddings_df_with_details.csv")
# embeddings_df = pd.read_csv("/home/batman/Desktop/py/cybersecurity-jina/data/eda_notebooks/unseen_128_preds.csv")
embeddings_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,dt,Label.1
0,0.0,30507650.0,0.0,0.0,0.0,128301070.0,0.0,110489750.0,59713990.0,0.0,...,27409.363636,21635.261127,92642,20839,1.000596e+07,14032.553854,10010275,9963654,2018-01-20 13:30:21,0
1,0.0,0.0,0.0,0.0,0.0,128371336.0,0.0,166059340.0,105106670.0,0.0,...,0.000000,0.000000,0,0,0.000000e+00,0.000000,0,0,2018-01-20 13:30:22,0


In [34]:
# index = DocumentArray()
# for i in range(len(embeddings_df)):
#     d = Document(
#         embedding=np.array(embeddings_df.iloc[i, :128].tolist()), 
#         tags={
#             "known_label" : embeddings_df.loc[i, "Label"],
#         }
#     )
#     index.append(d)
    
index = DocumentArray()
for i in range(len(embeddings_df)):
    d = Document(
        embedding=np.array(embeddings_df.iloc[i, :128].tolist()), 
        tags={
            "dt" : embeddings_df.loc[i, "dt"],
            "known_label" : embeddings_df.loc[i, "Label"],
            "port" : float(embeddings_df.loc[i, "Dst Port"]),
            "protocol" : float(embeddings_df.loc[i, "Protocol"]),
        }
    )
    index.append(d)
    
    

In [35]:
index.summary()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Length                 15000                         │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'tags', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   15000            False             │
│   id          ('str',)       15000            False             │
│   tags        ('dict',)      15000            False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

In [39]:
from config import configs
# embeddings from different day of data to find nearest neighbors in data already indexed
# dont want to cheat!
old_index = DocumentArray.load(configs["INDEX_PATH"]) 
# index.match(old_index, exclude_self=True) # or use find
index.summary()

╭───────────────────────── Documents Summary ──────────────────────────╮
│                                                                      │
│   Length                    15000                                    │
│   Homogenous Documents      True                                     │
│   Has nested Documents in   ('matches',)                             │
│   Common Attributes         ('id', 'tags', 'embedding', 'matches')   │
│   Multimodal dataclass      False                                    │
│                                                                      │
╰──────────────────────────────────────────────────────────────────────╯
╭──────────────────────── Attributes Summary ────────────────────────╮
│                                                                    │
│   Attribute   Data type         #Unique values   Has empty value   │
│  ────────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)      15000            False             │
│   id          ('str',)          15000            False             │
│   matches     ('MatchArray',)   15000            False             │
│   tags        ('dict',)         15000            False             │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯

In [37]:
known_attack = index.find({"tags__known_label" : {"$eq" : 1.0}})
old_index.find(known_attack[2])[0][0].tags

{'dt': '2018-01-23 02:56:03',
 'port': 53.0,
 'known_label': 0.0,
 'protocol': 17.0}

In [38]:
# NOTE: for find
y_test = [] # expected
yhat = [] # "predictions" aka nearest neighbor/brute force

for doc in index[:50]:
    # known
    if doc.tags.get("known_label") == 0.0:
        y_test.append(0.0)
    else:
        y_test.append(1.0)

    #if doc.matches[0].tags.get("known_label") == 0.0:
    if old_index.find(doc)[0][0].tags.get("known_label") == 0.0:
        yhat.append(0.0)
    else:
        yhat.append(1.0)


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat)
print('F1 score: %f' % f1)

Accuracy: 0.980000
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000


In [40]:
# NOTE: for match
y_test = [] # expected
yhat = [] # "predictions" aka nearest neighbor/brute force

for doc in index:
    # known
    if doc.tags.get("known_label") == 0.0:
        y_test.append(0.0)
    else:
        y_test.append(1.0)
    
    # these are nearest neighbor/brute force which is basically the "prediction" since we have embeddings stored in vector space as opposed to a forward pass through neural network or something
    if doc.matches[0].tags.get("known_label") == 0.0:
        yhat.append(0.0)
    else:
        yhat.append(1.0)


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat)
print('F1 score: %f' % f1)

Accuracy: 0.975867
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000


In [ ]:
# now do the same with weaviate

In [ ]:
da =  DocumentArray(
    storage='weaviate', config={'name': 'Persisted', 'host': 'localhost', 'port': 8080}
)

# # simulate; had problems using match with weavaite backend and getting it to persist (even when using as context manager ) so this is workaround
da = DocumentArray(da, copy=True)
da.match(da, exclude_self=True)

y_test = [] # expected
yhat = [] # "predictions" aka nearest neighbor/brute force

for doc in index:
    # known
    if doc.tags.get("known_label") == 0.0:
        y_test.append(0.0)
    else:
        y_test.append(1.0)
    
    # these are nearest neighbor/brute force which is basically the "prediction" since we have embeddings stored in vector space as opposed to a forward pass through neural network or something
    if doc.matches[0].tags.get("known_label") == 0.0:
        yhat.append(0.0)
    else:
        yhat.append(1.0)


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat)
print('F1 score: %f' % f1)

ModuleNotFoundError: No module named 'weaviate'